#Importing libraries

In [ ]:
%%capture
!pip install qwikidata
!pip install datasets
!pip install transformers
!pip install seqeval

In [ ]:
!wget https://zenodo.org/record/3234051/files/embeddings-l-model.vec?download=1

--2022-08-13 16:36:09--  https://zenodo.org/record/3234051/files/embeddings-l-model.vec?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3446609027 (3.2G) [application/octet-stream]
Saving to: ‘embeddings-l-model.vec?download=1.1’

embeddings-l-model. 100%[===================>]   3.21G  1.73MB/s    in 18m 17s 

2022-08-13 16:54:27 (3.00 MB/s) - ‘embeddings-l-model.vec?download=1.1’ saved [3446609027/3446609027]



In [ ]:
import pandas as pd
from qwikidata.sparql import return_sparql_query_results
import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
from nltk.tokenize.toktok import ToktokTokenizer
import numpy as np
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

from gensim.models.keyedvectors import KeyedVectors
wordvectors = KeyedVectors.load_word2vec_format('embeddings-l-model.vec?download=1.1', limit=100000)

#Reading datasets

##Raw datasets (.txt)

In [ ]:
def read_raw_dataset(dir):
  tok = []  #Aux list of tokens for current sentence
  bio = []  #Aux list of ner tags for current sentence
  df_list = []  #Final list with all the information

  with open(dir,'r') as file:
    while True:

      line = file.readline()

      if not line:
        break

      #Data cleaning
      if 'https' in line:
        continue

      #When reaching the end of a sentence, we append and restart tok and bio
      #We also check for non-empty sentences
      if line == '\n' and tok!=[] and bio!=[]:
        df_list.append([tok,bio])
        tok = []
        bio = []

      else:
        #Data cleaning
        if line.split(' ')[0]=='\n':
          continue

        #We add the token and ner_tag to the list
        tok.append(line.split(' ')[0])
        bio.append(line.split(' ')[-1].replace('\n',''))

  #Returning df_list to a dataframe
  return pd.DataFrame(df_list, columns=['tokens','ner_tags'])

In [ ]:
df = read_raw_dataset('train_spacy.txt')
df

,tokens,ner_tags
0,"[Cerramos, nuestra, querida, Radio, 😢, Nuestro...","[O, O, O, O, O, O, B-PROFESION, O, B-PROFESION..."
1,"[Desde, mañana, todos, los, programas, de, Rad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[Seguimos, al, aire, con, el, compromiso, de, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[#, OtroEscandalo, #, HastaCuando, #, Denuncia...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[¿, Es, necesario, entregar, nuestra, privacid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
12702,"[Se, os, ve, el, plumero, ...]","[O, O, O, O, O, O]"
12703,"[¡A, TOD@S, !]","[O, O, O]"
12704,"[Corríjanme, si, me, equivoco, ,, pero, somos,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
12705,"[Viendo, que, tenemos, 7, veces, más, casos, q...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


##Treated datasets (.csv)

In [ ]:
def read_treated_dataset(dir):
  df = pd.read_csv(dir)
  lambda x : x[2:-2].split("\', \'")

  df = df.apply(lambda x : x.str[2:-2])
  df = df.apply(lambda x: x.str.split("\', \'"))
  return df

In [ ]:
df = read_treated_dataset('base.csv')
df

,tokens,ner_tags
0,"[Cerramos, nuestra, querida, Radio, 😢, Nuestro...","[O, O, O, O, O, O, B-PROFESION, O, B-PROFESION..."
1,"[Desde, mañana, todos, los, programas, de, Rad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[Seguimos, al, aire, con, el, compromiso, de, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[#, OtroEscandalo, #, HastaCuando, #, Denuncia...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[¿, Es, necesario, entregar, nuestra, privacid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
12702,"[Se, os, ve, el, plumero, ...]","[O, O, O, O, O, O]"
12703,"[¡A, TOD@S, !]","[O, O, O]"
12704,"[Corríjanme, si, me, equivoco, ,, pero, somos,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
12705,"[Viendo, que, tenemos, 7, veces, más, casos, q...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


##Entity list

In [ ]:
def get_entities(df, tok):
  list1 = set()
  entity=''
  for i,j in zip(df['tokens'], df['ner_tags']):
    
    
    for i2,j2 in zip(i,j):
    
        if j2=='B-'+tok:
         
          entity =i2
          continue
        if j2=='I-'+tok:
          entity =entity +' '+i2
          continue
        else:
          if entity != '':
            list1.add(entity)
            entity=''

  return list1

In [ ]:
get_entities(df, 'PROFESION')

{'# AAMM',
 '# AgentesForestalesCM',
 '# Directores',
 '# Empresarios',
 '# GuardiaCivil',
 '# Medicos',
 '# PoliciaNacional',
 '# TES',
 '# abogados',
 '# artesanos',
 '# auxiliardevuelo',
 '# azafata',
 '# dentista',
 '# fotografo',
 '# internista',
 '# magistrado director',
 '# medicas',
 '# médico',
 '# reportero',
 '# runners',
 '# sanitarios',
 '# soldados',
 '# tattoist',
 '# temporeros',
 '# transportistas',
 '# tripulantedecabina',
 '@GuardiaCivil',
 '@guardiacivil',
 '@mossos',
 '@policia',
 'ALCALDE',
 'Abogados',
 'Agentes',
 'Agentes sociales',
 'Alcalde',
 'Alto Representante de UE',
 'Auxiliares',
 'Barrenderos',
 'Bomberos',
 'CAJERAS',
 'Capitán',
 'Cardenal',
 'Científicos',
 'Cirujanos',
 'Civiles',
 'Comisario',
 'Conductoras',
 'Consejera Medio Ambiente',
 'Coronel',
 'DJ',
 'De enfermería',
 'Dictador',
 'Director General de la OMS',
 'Director general de economía',
 'Directora General',
 'Directora del Instituto Salud',
 'Dirigentes',
 'Doctor',
 'Doctora',
 'Doc

#Halving the dataset

In [ ]:
def halve_dataset(df, ent_type):
  ent_list = get_entities(df,ent_type)
  half_ent_list = list(ent_list)[:round(len(ent_list)/2)]
  half_ent_list

  df_half_list = []

  for i,j in zip(df.loc[:,'tokens'], df.loc[:,'ner_tags']):
    if list(set(i) & set(half_ent_list))==[]:
      df_half_list.append([i,j])

  return pd.DataFrame(df_half_list, columns=['tokens','ner_tags'])

In [ ]:
df_half = halve_dataset(df, 'PROFESION')
df_half

,tokens,ner_tags
0,"[Desde, mañana, todos, los, programas, de, Rad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Seguimos, al, aire, con, el, compromiso, de, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[#, OtroEscandalo, #, HastaCuando, #, Denuncia...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[¿, Es, necesario, entregar, nuestra, privacid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[La, privacidad, en, tiempos, de, coronavirus,...","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
11923,"[Se, os, ve, el, plumero, ...]","[O, O, O, O, O, O]"
11924,"[¡A, TOD@S, !]","[O, O, O]"
11925,"[Corríjanme, si, me, equivoco, ,, pero, somos,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
11926,"[Viendo, que, tenemos, 7, veces, más, casos, q...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


#Wikidata Augmentation

In [ ]:
#Wikidata query that searches the "child" of an entity given a relation in a language with a word limit
def search_child(entity, relation, lang, limit):
    res = []
    try:
        query = '''
        select ?label
        where{
            ?item wdt:#RELATION wd:#ENTITY .
            bind(SHA512(concat(str(rand()), str(?item))) as ?random) .
            ?item rdfs:label ?label filter (lang(?label) = "#LANG").
        }
        order by ?random
        limit #LIMIT
        '''
        query = query.replace('#ENTITY',entity).replace('#LIMIT',str(limit)).replace('#RELATION',relation).replace('#LANG',lang)

        query_res = return_sparql_query_results(query)

        for i in query_res['results']['bindings']:
            res.append(i['label']['value'])

        return res

    except Exception as e:
        print("Exception:",e)
        return None



#Gets a profession and generates a list with the tokens and ner_tags corresponding
def treat_prof_list(str):
  tok = str.split(' ')
  bio = ['B-PROFESION'] + (['I-PROFESION']*(len(tok)-1))
  return([tok, bio])



#Generates a list with 1000 professions tokenized and with ner_tags
def gen_prof():
  prof_list = search_child('Q28640','P31','es',1000)
  return list(map(treat_prof_list, prof_list))



def find_prof_sentences(num):
  toktok = ToktokTokenizer()
  lang = 'es'
  res = []
  temp_res = []
  rep = round((num**(1/2)/2))

  while True:

    proflist = search_child('Q28640','P31','es',num*3)
    if proflist==None or proflist==[]:
      continue

    for wrd in proflist:
      text = []
      temp_res = []
      elem = wrd.replace(' ','_').capitalize()

      response = requests.get(f'https://{lang}.wikipedia.org/wiki/{elem}')
      soup = BeautifulSoup(response.content, 'html.parser')
      
      if response.status_code != 200:
        continue

      for paragraph in soup.find_all('p'):
        p = re.sub('[\(\[].*?[\)\]]', '', paragraph.text).strip()
        if p.find('may refer to:')==-1 and p.find('\\')==-1 and p.find('{')==-1:
          text += p.split('.')

      for count,i in enumerate(text):
        if i.find(wrd)!=-1:
          temp_res.append(i.strip()+'.')
          if count>rep:
            break

      if temp_res != []:
        wrd_first = wrd.split(' ')[0]
        wrd_next = wrd.split(' ')[1:]

        tok_temp_res = list(map(lambda x : toktok.tokenize(x), temp_res))
        bio_temp_res = []
        for i in tok_temp_res:
          bio_temp_res.append(list(map(lambda x :'B-PROFESION' if (wrd_first in x) else ('I-PROFESION' if any(item in x for item in wrd_next) else 'O'), i)))
        
        for i,j in zip(tok_temp_res, bio_temp_res):
          res.append([i,j])
      
      if (len(res) > num):
        return res



def get_wikipedia_aug_dataset(n):
  return pd.DataFrame(find_prof_sentences(n), columns=['tokens','ner_tags'])

In [ ]:
wikipedia_aug_df = get_wikipedia_aug_dataset(10)
wikipedia_aug_df

,tokens,ner_tags
0,"[Según, Erberto, Lo, Bue, ,, el, tibetólogo, h...","[O, O, O, O, O, O, B-PROFESION, O, O, O, O, O,..."
1,"[En, el, 2014, ,, el, matrimonio, formado, por...","[O, O, O, O, O, O, O, O, O, O, O, B-PROFESION,..."
2,"[En, el, mundo, de, la, música, clásica, ,, el...","[O, O, O, I-PROFESION, O, O, O, O, O, B-PROFES..."
3,"[Un, tutor, es, una, persona, que, asiste, a, ...","[O, B-PROFESION, O, O, O, O, O, O, O, O, O, O,..."
4,"[En, la, Grecia, Antigua, ,, la, tutoría, inic...","[O, O, O, O, O, O, B-PROFESION, O, O, O, O, O,..."
5,"[Los, tutores, operaban, en, todo, tipo, de, c...","[O, B-PROFESION, O, O, O, O, O, O, O, O, O, O,..."
6,"[Dada, la, laboriosidad, del, proceso, de, tra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
7,"[Un, técnico, de, mantenimiento, de, aeronaves...","[O, B-PROFESION, I-PROFESION, I-PROFESION, I-P..."
8,"[El, director, de, operaciones, o, jefe, de, o...","[O, O, I-PROFESION, I-PROFESION, O, B-PROFESIO..."
9,"[El, jefe, de, operaciones, es, uno, de, los, ...","[O, B-PROFESION, I-PROFESION, I-PROFESION, O, ..."


##Word Embedding Filter 

In [ ]:
seed_wrds = ['presidentes', 'economista', 'consejera', 'investigadores', 'abogado', 'deportistas', 'gestores', 'político', 'presidenta', 'Rey', 'jueza', 'Capitán', 'dictador', 'alcalde', 'futbolista', 'asesores', 'sanitaria', 'enfermeras', 'médicos', 'Policía', 'abogados', 'Científicos', 'primer', 'vicepresidente', 'empresarios', 'periodista', 'conductores', 'Dirigentes', 'Ministra', 'policías', 'arzobispo', 'Presidente', 'Policías', 'empresario', 'director', 'Sanitarios', 'sanitarios', 'doctor', 'artistas', 'ladrón', 'médica', 'policia', 'presidente', 'asesor', 'juez', 'Ministros', 'jueces', 'alcaldes', 'Bomberos', 'policía', 'fundador', 'diputada', 'forense', 'investigador', 'profesionales', 'futbolistas', 'militares', 'rey', 'políticos', 'diputado', 'canciller', 'periodistas', 'médico', 'taxistas', 'jugadores', 'guardia', 'enfermera', 'portavoz', 'Guardia', 'cajera', 'diputados', 'consejero', 'ministro', 'Cardenal', 'Reyes', 'científicos', 'ministros', 'agentes', 'Presidenta', 'ministra', 'vicepresidenta', 'enfermero']
seed_wrds = list(map(lambda x: x.lower(), seed_wrds))

def getSim(x):
  res = -1
  try:
    res = wordvectors.n_similarity(seed_wrds,x)
  except:
    try:
      res = wordvectors.n_similarity(seed_wrds,x.split(' ')[0])
    except:
      res = 0
  return res


def gen_prof_embedding(num):
  prof_list = search_child('Q28640','P31','es',num)
  prof_list_emb = list(map(getSim, prof_list))
  filt_prof_list = list(filter(lambda x: prof_list_emb[prof_list.index(x)]>np.percentile(prof_list_emb, 80), prof_list))
  return filt_prof_list

In [ ]:
def find_prof_sentences_embedding(num):
  toktok = ToktokTokenizer()
  lang = 'es'
  res = []
  temp_res = []
  rep = round((num**(1/2)/2))

  while True:

    proflist = gen_prof_embedding(num*3)
    if proflist==None or proflist==[]:
      continue

    for wrd in proflist:
      text = []
      temp_res = []
      elem = wrd.replace(' ','_').capitalize()

      response = requests.get(f'https://{lang}.wikipedia.org/wiki/{elem}')
      soup = BeautifulSoup(response.content, 'html.parser')
      
      if response.status_code != 200:
        continue

      for paragraph in soup.find_all('p'):
        p = re.sub('[\(\[].*?[\)\]]', '', paragraph.text).strip()
        if p.find('may refer to:')==-1 and p.find('\\')==-1 and p.find('{')==-1:
          text += p.split('.')

      for count,i in enumerate(text):
        if i.find(wrd)!=-1:
          temp_res.append(i.strip()+'.')
          if count>rep:
            break

      if temp_res != []:
        wrd_first = wrd.split(' ')[0]
        wrd_next = wrd.split(' ')[1:]

        tok_temp_res = list(map(lambda x : toktok.tokenize(x), temp_res))
        bio_temp_res = []
        for i in tok_temp_res:
          bio_temp_res.append(list(map(lambda x :'B-PROFESION' if (wrd_first in x) else ('I-PROFESION' if any(item in x for item in wrd_next) else 'O'), i)))
        
        for i,j in zip(tok_temp_res, bio_temp_res):
          res.append([i,j])
      
      if (len(res) > num):
        return res



def get_wikipedia_aug_dataset_embedding(n):
  return pd.DataFrame(find_prof_sentences(n), columns=['tokens','ner_tags'])

In [ ]:
wikipedia_aug_df_embedding = get_wikipedia_aug_dataset_embedding(10)
wikipedia_aug_df_embedding

,tokens,ner_tags
0,"[Asimismo, ,, un, piloto, de, rally, puede, in...","[O, O, O, B-PROFESION, I-PROFESION, I-PROFESIO..."
1,"[El, hombre, de, armas, fue, un, soldado, de, ...","[O, B-PROFESION, I-PROFESION, I-PROFESION, O, ..."
2,"[No, obstante, ,, el, desarrollo, del, uso, de...","[O, O, O, O, I-PROFESION, I-PROFESION, O, I-PR..."
3,"[El, término, librero, puede, hacer, referenci...","[O, O, B-PROFESION, O, O, O, O, O]"
4,"[Un, vaquero, es, el, encargado, de, las, tare...","[O, B-PROFESION, O, O, O, O, O, O, O, O, O, O,..."
5,"[También, es, llamado, en, inglés, buckaroo, p...","[O, O, O, O, O, O, O, O, O, O, O, B-PROFESION,..."
6,"[El, término, viajero, puede, referirse, ,, en...","[O, O, B-PROFESION, O, O, O, O, O, O, O, O]"
7,"[Un, artista, callejero, es, una, persona, que...","[O, B-PROFESION, I-PROFESION, O, O, O, O, O, O..."
8,"[En, la, Argentina, aún, existen, algunas, esc...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9,"[El, trabajo, de, un, relojero, no, se, ha, vi...","[O, O, O, O, B-PROFESION, O, O, O, O, O, O, O,..."


#Concat two databases

In [ ]:
def concat_databases(df1, df2, max):
  df_concat = pd.concat([df1,df2])
  df_res = df_concat.head(max)

  return df_res

In [ ]:
concat_databases(df_half, wikipedia_aug_df, 12707)

,tokens,ner_tags
0,"[Desde, mañana, todos, los, programas, de, Rad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Seguimos, al, aire, con, el, compromiso, de, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[#, OtroEscandalo, #, HastaCuando, #, Denuncia...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[¿, Es, necesario, entregar, nuestra, privacid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[La, privacidad, en, tiempos, de, coronavirus,...","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
100,"[La, labor, del, fotógrafo, publicitario, es, ...","[O, O, O, B-PROFESION, I-PROFESION, O, O, O, O..."
101,"[El, fotógrafo, publicitario, tiene, que, logr...","[O, B-PROFESION, I-PROFESION, O, O, O, O, O, O..."
102,"[La, especialización, del, fotógrafo, publicit...","[O, O, O, B-PROFESION, I-PROFESION, O, O, O, O..."
103,"[Al, finalizar, la, Segunda, Guerra, Mundial, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


#Export database

In [ ]:
df_res.to_csv('./first_dataset.csv',index=False)
pd.read_csv('./first_dataset.csv')